In [25]:
import pandas as pd

In [26]:
df = pd.read_csv("spotify_millsongdata.csv")

In [27]:
df.head(5)

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \r\nA..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \r\nTouch me gen..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \r\nWhy I had...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [28]:
df.tail(5)

,artist,song,link,text
57645,Ziggy Marley,Good Old Days,/z/ziggy+marley/good+old+days_10198588.html,Irie days come on play \r\nLet the angels fly...
57646,Ziggy Marley,Hand To Mouth,/z/ziggy+marley/hand+to+mouth_20531167.html,Power to the workers \r\nMore power \r\nPowe...
57647,Zwan,Come With Me,/z/zwan/come+with+me_20148981.html,all you need \r\nis something i'll believe \...
57648,Zwan,Desire,/z/zwan/desire_20148986.html,northern star \r\nam i frightened \r\nwhere ...
57649,Zwan,Heartsong,/z/zwan/heartsong_20148991.html,come in \r\nmake yourself at home \r\ni'm a ...


In [29]:
df.shape

(57650, 4)

In [30]:
df.isnull().sum()

artist    0
song      0
link      0
text      0
dtype: int64

In [31]:
df =df.sample(5000).drop('link', axis=1).reset_index(drop=True)

In [32]:
df.head(10)

,artist,song,text
0,Kiss,Ozone,I'm the kind of guy who likes feelin' high \r...
1,Passenger,Carnival Diaries,What would you sacrifice for reality? \r\nIs ...
2,Pearl Jam,Break On Through,You know the day destroys the night \r\nNight...
3,Janis Joplin,No Reason For Livin',( from the soundtrack of the motion picture 'j...
4,John Legend,Love The Way It Should Be,"Whoa, oh, yeah \r\n \r\nTime is gettin' hard..."
5,Billie Holiday,Cheek To Cheek,"Heaven, I'm in heaven \r\nAnd my heart beats ..."
6,Neil Diamond,Glory Road,"Wearin' my high boots, \r\nGot all my world l..."
7,Bette Midler,"Together, Wherever We Go","Wherever we go, whatever we do \r\nWe're gonn..."
8,Deep Purple,You Can't Do It Right,Have you ever thought of the feeling \r\nI ge...
9,John Denver,Healing Time,Let the mountains talk \r\nLet the rivers run...


In [33]:
df['text'][0]

"I'm the kind of guy who likes feelin' high  \r\nFeelin' high and dry, and I really like to fly  \r\nI'm your kinda guy, girl I'm not too shy  \r\nAnd I want you to fly, so I think you oughta try  \r\nOzone, ozone, ozone, ozone  \r\n  \r\nI'm your kinda guy and I'm not too shy  \r\nFeelin' high and dry, and I want you to be mine  \r\nSo I think it's time, girl, to start feeling fine  \r\nFeeling good all the time, so I think you oughta try  \r\n  \r\nOzone, ozone, ozone, ozone  \r\n  \r\nNow I'm the kinda guy who likes getting high  \r\nGetting high and dry, and I do it all the time  \r\nIf you think it's a crime and you don't wanna try  \r\nSomething that'll make you high, then I think I'll say goodbye  \r\n  \r\n'Cos I'm the kind of guy who likes getting high  \r\nGetting high and dry, and I do it all the time  \r\nNow I'm much too high, I am high and dry  \r\nAnd I'm feelin' fine, I'm feelin' good all of the time... (to fade)\r\n\r\n"

In [34]:
# df = df.sample(5000)

In [35]:
df.shape

(5000, 3)

Text Cleaning/ Text Preprocessing

In [36]:
df['text'] = df['text'].str.lower().replace(r'^\w\s', ' ').replace(r'\n', ' ', regex = True)

In [37]:
import nltk
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

def tokenization(txt):
    tokens = nltk.word_tokenize(txt)
    stemming = [stemmer.stem(w) for w in tokens]
    return " ".join(stemming)


In [38]:
# df['text'] = df['text'].apply(lambda x: tokenization(x))

In [39]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [40]:
tfidvector = TfidfVectorizer(analyzer='word',stop_words='english')
matrix = tfidvector.fit_transform(df['text'])
similarity = cosine_similarity(matrix)

In [41]:
similarity[0]

array([1.        , 0.02612778, 0.02053917, ..., 0.01224924, 0.        ,
       0.00478539])

In [42]:
df[df['song'] == 'Crying Over You']

,artist,song,text
3453,UB40,Crying Over You,crying over you in the morning \r crying over...


In [43]:
def recommendation(song_df):
    idx = df[df['song'] == song_df].index[0]
    distances = sorted(list(enumerate(similarity[idx])),reverse=True,key=lambda x:x[1])
    
    songs = []
    for m_id in distances[1:21]:
        songs.append(df.iloc[m_id[0]].song)
        
    return songs

In [44]:
print(df)

               artist                       song   
0                Kiss                      Ozone  \
1           Passenger           Carnival Diaries   
2           Pearl Jam           Break On Through   
3        Janis Joplin       No Reason For Livin'   
4         John Legend  Love The Way It Should Be   
...               ...                        ...   
4995  George Harrison             World Of Stone   
4996      Nicki Minaj                  Automatic   
4997  George Harrison        Roll Over Beethoven   
4998      Cole Porter          Miss Otis Regrets   
4999             Cher            It's Gonna Rain   

                                                   text  
0     i'm the kind of guy who likes feelin' high  \r...  
1     what would you sacrifice for reality?  \r is i...  
2     you know the day destroys the night  \r night ...  
3     ( from the soundtrack of the motion picture 'j...  
4     whoa, oh, yeah  \r   \r time is gettin' harder...  
...                        

In [45]:
recommendation('Crying Over You')

['Crazy For You',
 'Blue Eyes Crying In The Rain',
 'I Will Always Want You',
 "Everybody's Cryin Mercy",
 'Leaving',
 'Boy',
 'Wrong',
 "Small Y'all",
 'Solitude',
 "Don't Know How (Not To Love You)",
 'Even A Fool Would Let Go',
 "When You're Gone",
 'Hero Of The Day',
 'Roulette',
 'I Was A Lover, A Leader Of Men',
 'Loveless',
 'From The Balcony',
 'Extraordinary Girl',
 "Don't Stop Now",
 'Saturday Night']

In [46]:
import pickle
pickle.dump(similarity,open('similarity.pkl','wb'))
pickle.dump(df,open('df.pkl','wb'))